In [ ]:
# Modeling

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pickle
from sklearn.feature_selection import VarianceThreshold
from fuzzywuzzy import process
from sklearn.model_selection import GridSearchCV
import time
import warnings
warnings.filterwarnings('ignore')

#Connecting the driver
from google.colab import drive

drive.mount('/content/drive/')

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Mounted at /content/drive/


In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
def accuracy_measure(y_test, y_pred):
    sensitivity_overall = recall_score(y_test, y_pred, average='weighted')
    print('Sensitivity Overall Data:',sensitivity_overall)

    precision_overall = precision_score(y_test, y_pred, average='weighted')
    print('Precision Overall Data:',precision_overall)

    f1_overall = f1_score(y_test, y_pred, average='weighted')
    print('F1 Score Overall Data:',f1_overall)

    # Cohen's Kappa statistic
    kappa = cohen_kappa_score(y_test, y_pred)
    print("Cohen's Kappa:", kappa)

    # Matthews Correlation Coefficient (MCC)
    mcc = matthews_corrcoef(y_test, y_pred)
    print("Matthews Correlation Coefficient (MCC):", mcc)

    # Compute log loss
    # logloss = log_loss(y_test, y_pred)
    # print(f"Log Loss: {logloss}")

    # # AUC and ROC
    # fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    # roc_auc = auc(fpr, tpr)

    # # Plot ROC curve
    # plt.figure(figsize=(8, 6))
    # plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    # plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title('Receiver Operating Characteristic (ROC) Curve')
    # plt.legend(loc='lower right')
    # plt.show()

    # # Visualize predicted probabilities distribution
    # plt.figure(figsize=(8, 6))
    # sns.histplot(y_pred, bins=20, kde=True, color='skyblue')
    # plt.xlabel('Predicted Probabilities')
    # plt.ylabel('Frequency')
    # plt.title('Distribution of Predicted Probabilities')
    # plt.show()

    # # Confusion Matrix
    # cm = confusion_matrix(y_test, y_pred)
    # print(cm)

    # # Printing Confusion Matrix calsswise
    # unique_classes = np.unique(y_test)

    # # Initialize variables to store metrics
    # total_tp = 0
    # total_tn = 0
    # total_fp = 0
    # total_fn = 0

    # Iterate over each class
    # for cls in unique_classes:
    #     idx = (y_test == cls)
    #     true_positive = np.sum((y_pred[idx] == cls))
    #     true_negative = np.sum((y_pred != cls) & (y_test != cls))
    #     false_positive = np.sum((y_pred == cls) & (y_test != cls))
    #     false_negative = np.sum((y_pred != cls) & (y_test == cls))

    #     total_tp += true_positive
    #     total_tn += true_negative
    #     total_fp += false_positive
    #     total_fn += false_negative

    #     print(f"\nClass {cls} Metrics:")
    #     print(f"True Positive (TP): {true_positive}")
    #     print(f"True Negative (TN): {true_negative}")
    #     print(f"False Positive (FP): {false_positive}")
    #     print(f"False Negative (FN): {false_negative}")

    # # Consolidated results
    # print("\nConsolidated Results:")
    # print(f"Total True Positive (TP): {total_tp}")
    # print(f"Total True Negative (TN): {total_tn}")
    # print(f"Total False Positive (FP): {total_fp}")
    # print(f"Total False Negative (FN): {total_fn}")


Gradient boost without dimensionality reduction

---



In [ ]:
df_training = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/training_dataset_final.csv")
X =df_training.drop(columns=['disease'])
y =df_training['disease']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# Print the shapes of the resulting sets
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

Training set: (8668, 404) (8668,)
Validation set: (1858, 404) (1858,)
Test set: (1858, 404) (1858,)


In [13]:
print('####### Gradient boost without dimensionality reduction')
classifier = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,subsample=0.8,random_state=10)
time_now=time.time()
classifier.fit(X_train, y_train)
print('time taken: ',time.time()-time_now)

####### Gradient boost without dimensionality reduction
time taken:  1953.2482314109802


In [14]:
# Make predictions on the test set
y_pred_basline = classifier.predict(X_test)
accuracy_baseline = accuracy_score(y_test, y_pred_basline)
print("Test Accuracy Score:", accuracy_baseline)
accuracy_measure(y_test, y_pred_basline)

Test Accuracy Score: 0.8972012917115177
Sensitivity Overall Data: 0.8972012917115177
Precision Overall Data: 0.9053183481209972
F1 Score Overall Data: 0.8970338154218543
Cohen's Kappa: 0.8962877236973424
Matthews Correlation Coefficient (MCC): 0.8963854372972909


### Using various dimensionality reduction

In [15]:
def low_var():
    df_low_variance = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/low_variance_features.csv")
    df_low_variance= df_low_variance.fillna(0)
    df_low_variance.isna().any()
    X = df_low_variance.drop('disease', axis=1)
    y = df_low_variance['disease']
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X,y,X_train,y_train,X_test,X_val,y_test, y_val

def rand_forest():
    df_training = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/training_dataset_final.csv")
    df_feature = pd.read_csv("/content/drive/MyDrive/DAP/python_scripts/feature_importance_df.csv")
    df_feature= df_feature.fillna(0)
    df_training= df_training.fillna(0)

    feature_list=(df_feature['Feature'].head(100)).tolist()
    df_training_important_Features = df_training[feature_list]
    df_training_important_Features['disease'] = df_training['disease']
    df_training_important_Features.info()
    X =df_training_important_Features.drop(columns=['disease'])
    y =df_training_important_Features['disease']
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    return X,y,X_train,y_train,X_test,X_val,y_test, y_val






# Gradient boost

In [16]:
def gradient_boost(X,y,X_train,y_train,X_test,X_val,y_test, y_val,pickle_file_name):
    classifier = GradientBoostingClassifier()
    classifier.fit(X_train, y_train)

    # Make predictions on the test set
    print('####### Gradient boost without specifying any parameters')
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Test Accuracy Score:", accuracy)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1 Score:", f1)
    # ----------------------------------------------------------------------

    # Performing Grid Search using hyperparamters
    print('\n####### Gradient boost with grid search')
    param_test1 = {'n_estimators':range(40,81,10),'max_features':["log2","sqrt"]}
    clf = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,subsample=0.8,random_state=10),
    param_grid = param_test1, n_jobs=4)
    best_clf = clf.fit(X_train, y_train)

    cv_results = pd.DataFrame(best_clf.cv_results_)
    print (f'Accuracy: {best_clf.score(X,y):.3f}')
    print(best_clf)
    best_params = best_clf.best_params_
    print("Best hyperparameters:", best_clf.best_params_)
    print("Best model:", best_clf.best_estimator_)

    best_params = best_clf.best_params_
    # Create a Gradient boost model with the best hyperparameters
    best_gb_model = GradientBoostingClassifier(**best_params, learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,subsample=0.8,random_state=10)

    # Train the model on the training data
    model = best_gb_model.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = best_gb_model.predict(X_test)

    # Evaluate the performance of the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy score:", accuracy)
    print('\n####### Cross Validation')
    cv_scores = cross_val_score(best_gb_model, X_train, y_test, cv=5)
    print('Scores from cross-validation is: ', cv_scores)
    print('Average accuracy from cross-validation is: {}'.format(np.mean(cv_scores)))

    accuracy_measure(y_test, y_pred)
    # ----------------------------------------------------------------------

    ### Validating the model
    print('\n####### Validating the model')
    predicted_class_validation = best_gb_model.predict(X_val)
    print("Predicted class:", predicted_class_validation)

    accuracy = accuracy_score(y_val, predicted_class_validation)
    print("Accuracy on the validation set:", accuracy)
    accuracy_measure(y_val, predicted_class_validation)

    ### Saving the model with best parameters logistic regression
    print( "\n####### Saving the model with best parameters gradient boost" )
    # Save the model to a file using pickle
    with open(pickle_file_name, 'wb') as file:
        pickle.dump(model, file)


Gradient Boost after dimensionality reduction with Low Filter Variance

In [17]:
X,y,X_train,y_train,X_test,X_val,y_test, y_val = low_var()
# Print the shapes of the resulting sets
print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

Training set: (8668, 104) (8668,)
Validation set: (1858, 104) (1858,)
Test set: (1858, 104) (1858,)


In [18]:
with open(r'low_var_features.txt', 'w') as fp:
    for item in X_train:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
gradient_boost(X,y,X_train,y_train,X_test,X_val,y_test, y_val,'gradient_boost_model_lv.pkl')

####### Gradient boost without specifying any parameters
Test Accuracy Score: 0.9413347685683531
F1 Score: 0.9394152447602064

####### Gradient boost with grid search


Gradient Boost after dimensionality reduction with Random Forest

In [ ]:
X,y,X_train,y_train,X_test,X_val,y_test, y_val = rand_forest()

In [ ]:
with open(r'rand_for_features.txt', 'w') as fp:
    for item in X_train:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [ ]:
gradient_boost(X,y,X_train,y_train,X_test,X_val,y_test, y_val,'gradient_boost_model_rd.pkl')

In [ ]:
with open('gradient_boost_rd.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

### Predicting the Disease

In [ ]:
# getting primitive care
df_primitive_care = pd.read_csv("prec_t.csv")
diagnose_list = df_primitive_care['diagnose'].tolist()
df_primitive_care.head(3)


In [ ]:
def predicting_disease(syp,diagnose_list,feature_names):
    columns_list=feature_names.copy()
    for col in syp:
        if col not in feature_names:
            #print(col)
            syp.remove(col)

    temp_df = pd.DataFrame(0, index=[0], columns=columns_list)
    temp_df.loc[0, syp] = 1
    print(syp)
    print(temp_df)
    predicted_class_1 = loaded_model.predict(temp_df)
    #print("Predicted class:", predicted_class_1)
    # getting primitive care
    predicted_class_1 = str(predicted_class_1)
    diagnose_list = [str(item) for item in diagnose_list]
    best_match, similarity_score = process.extractOne(predicted_class_1, diagnose_list)
    threshold = 70
    if similarity_score >= threshold:
        print(f"Best match for '{predicted_class_1}': '{best_match}' with similarity score: {similarity_score}")
        primitive_care= df_primitive_care[df_primitive_care['diagnose']==best_match]
        primitive_care = primitive_care['pid'].iloc[0]
        message = f"Based on the symptoms you entered, there are chances that you are suffering from **{predicted_class_1}**.\n"\
          f"\nThe primitive care we would suggest is **{primitive_care}** \nThis recommendation is based on the symptoms you provided.\n "\
        "However, please note that this is not a substitute for professional medical advice. We strongly recommend consulting a healthcare professional "\
          "for further clarification and a proper diagnosis. Your health is important, and a medical professional can provide personalized guidance and care."
        print(message)

    else:
        print(f" Diseases Predicted '{predicted_class_1}' in the is not monitored.")





In [ ]:
feature_names = X_train.columns.tolist()
syp = ['facial paresis','muscle twitch','chill','fever','cough','decreased body weight']

In [ ]:
predicting_disease(syp,diagnose_list,feature_names)

In [ ]:
syp = ['abscess bacterial','muscle twitch','chill','fever','barking cough']
predicting_disease(syp,diagnose_list,feature_names)
# in this case "abscess bacterial" is not the correct syp